In [43]:
# Make a file `.env` and put your OPEN_AI_KEY there.

import os
from dotenv import load_dotenv
load_dotenv()
OPEN_AI_KEY = os.environ.get('OPEN_AI_KEY')

In [44]:
import requests
import openai
import json
from agent_tools.base import Tool, action, observation

openai.api_key = OPEN_AI_KEY
client = openai.OpenAI(api_key=OPEN_AI_KEY)

In [45]:
from agentscript import Interpreter, Invocation, InvokationStatus
from deepthread import Thread, Message

In [46]:
class WeatherLogger(Tool):
    def __init__(self):
        super().__init__()
        self.log_file = "weather.txt"

    @action
    def log(self, message: str):
        """Logs a message to the log file."""
        with open(self.log_file, "a") as file:
            file.write("***\n" + message + "\n")

    @observation
    def weather(self, location: str) -> str:
        """Checks the current weather from the internet using wttr.in."""
        weather_api_url = f"http://wttr.in/{location}?format=%l:+%C+%t"
        try:
            response = requests.get(weather_api_url)
            return response.text
        except requests.exceptions.HTTPError as err:
            return f"HTTP Error: {err}"
        except requests.exceptions.RequestException as e:
            return f"Error: Could not retrieve weather data. {e}"
        
    def close(self):
        """Close the WeatherLogger tool and release any resources."""
        # Since there are no resources to release in this simple example, pass is used.
        pass


In [47]:
interpreter = Interpreter(tools=[WeatherLogger()])

interpreter.execute(
    """
    We need check the weather <Invoke tool="WeatherLogger" action="weather" parameters={"location": "New York"} />
    """
)
interpreter.invocations()[0].result

'New York: Overcast +6°C'

In [48]:
thread = Thread(owner_id="mariya")
thread.add_participant("system")
thread.add_participant("assistant")
thread.add_participant("user")
thread.post("system", "You are a weather assistant")
thread.post("user", "We need to check the weather in New York")
messages = thread.messages()


In [49]:
def get_messages(thread: Thread):
    return [{"role": message.user_id, "content": message.text} for message in thread.messages()]

def ask_ai(thread):
    messages = get_messages(thread)
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=messages,
        max_tokens=1000
    )
    print(f"Full response: {response}")
    assistant_reply = response.choices[0].message.content
    print(f"Actual reply: {assistant_reply}")
    return assistant_reply

def execute_step(reply, interpreter: Interpreter):
    interpreter.execute(reply)
    result = interpreter.invocations()[-1].result
    print(f"Result of execution: {result}")
    return result



In [50]:
schema = WeatherLogger().json_schema()

In [51]:
system_message = f"""
You are a helpful poetical AI. 
You can check the weather, write a short poem to describe it, and log the poem. 
You have the following tools available to you in the tool "WeatherLogger": 

{str(schema)}

When you respond, wrap the answer to the following object:

<Invoke tool="NameOfTheTool" action="name_of_the_action" parameters={{"param1": "value1", "param2": "value2"}} />

For example, if you want to log the weather in Thessaloniki, you would respond with:

<Invoke tool="WeatherLogger" action="weather" parameters={{"location": "Thessaloniki"}} />

Feel free to add comments about why you do so.
"""

task = "Check the weather in Paris, describe it, and log the result."

In [52]:
interpreter = Interpreter(tools=[WeatherLogger()])
thread = Thread(owner_id="user")
thread.add_participant("system")
thread.add_participant("assistant")
thread.add_participant("user")


In [53]:
thread.post("system", system_message)
thread.post("user", task)

In [54]:
advice = ask_ai(thread)
thread.post("assistant", advice)
result = execute_step(advice, interpreter)
thread.post("user", result)


Full response: ChatCompletion(id='chatcmpl-8wYR6NYTjItnDrXCWntA65Qf1OtXH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<Invoke tool="WeatherLogger" action="weather" parameters={"location": "Paris"} />', role='assistant', function_call=None, tool_calls=None))], created=1708966088, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_91aa3742b1', usage=CompletionUsage(completion_tokens=19, prompt_tokens=273, total_tokens=292))
Actual reply: <Invoke tool="WeatherLogger" action="weather" parameters={"location": "Paris"} />
Result of execution: Paris: Light rain, rain +7°C


In [55]:
advice = ask_ai(thread)
thread.post("assistant", advice)
result = execute_step(advice, interpreter)
if result:
    thread.post("user", result)
else:
    thread.post("user", "Thank you.")

Full response: ChatCompletion(id='chatcmpl-8wYR8UYDMTdzivfP4FzLOKVjfZM0a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In Paris the clouds weep gently, a shroud,\nA veil of light rain, whispers not loud.\nAmidst the brisk air, where breaths can be seen,\nThe city of lights, in a wash of serene.\n\nA temperature mild, at seven degrees,\nWhere warmth wraps the chill in a soft tease.\nIn this scene of silence, amid drops\' ballet,\nParis whispers tales, in the rain\'s soft play.\n\n<Invoke tool="WeatherLogger" action="log" parameters={"message": "Paris: Light rain, a serene scene with a whisper of chill at +7°C."} />', role='assistant', function_call=None, tool_calls=None))], created=1708966090, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_91aa3742b1', usage=CompletionUsage(completion_tokens=123, prompt_tokens=309, total_tokens=432))
Actual reply: In Paris the clouds weep gently, a shroud,
A veil of l

In [56]:
for m in thread.messages():
    print(m.user_id, "->\n", m.text, "\n---")

system ->
 
You are a helpful poetical AI. 
You can check the weather, write a short poem to describe it, and log the poem. 
You have the following tools available to you in the tool "WeatherLogger": 

[{'name': 'log', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string'}}, 'required': ['message']}, 'description': 'Logs a message to the log file.'}, {'name': 'weather', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string'}}, 'required': ['location']}, 'description': 'Checks the current weather from the internet using wttr.in.'}]

When you respond, wrap the answer to the following object:

<Invoke tool="NameOfTheTool" action="name_of_the_action" parameters={"param1": "value1", "param2": "value2"} />

For example, if you want to log the weather in Thessaloniki, you would respond with:

<Invoke tool="WeatherLogger" action="weather" parameters={"location": "Thessaloniki"} />

Feel free to add comments about why you do so.
 
---
user ->
 Check